# Step 6: Advanced RAG Evaluation Using RAGAS

## 0) Configs
- From Google Drive, locate Step 2 and 5 notebooks first
- Store results in step6_ragas_results.csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# Change all paths to your own path before reproduction
STEP2_NOTEBOOK = "/content/drive/My Drive/Colab Notebooks/Naive_RAG_Milvus_Step2.ipynb"
STEP5_NOTEBOOK = "/content/drive/My Drive/Colab Notebooks/Advanced_RAG_Step5.ipynb"

N_SAMPLES = 50         # For sanity test, set sample size to 50 first
SAVE_CSV = "/content/drive/MyDrive/NLX_LLM_Project_2/step6_ragas_results.csv"


## 1) Install packages

In [ ]:
!pip -q install datasets evaluate pandas ragas "langchain-openai>=0.1.7" transformers sentence-transformers faiss-cpu

import os, numpy as np, pandas as pd
from datasets import load_dataset, Dataset
from typing import List, Dict
from pathlib import Path

# RAGAS
from ragas import evaluate as ragas_evaluate
from ragas.metrics import (
    faithfulness, answer_relevancy, context_precision, context_recall
)

# Optional: if OpenAI key available in env var, import & use ChatGPT 4o-mini
#HAS_OPENAI = bool(os.getenv("OPENAI_API_KEY"))
#print("OpenAI key detected:", HAS_OPENAI)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.3/284.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.5/155.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.0/352.0 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source

In [ ]:
!pip install pymilvus[milvus_lite]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 21.4 MB/s eta 0:00:00


## 2) Import and run naive/advanced RAG models in Step 2 and 5 notebooks
First check absolute paths for previous documents to avoid conversion error.

In [ ]:
import os

# Assuming your notebooks are in the 'Colab Notebooks' folder directly under 'My Drive'
notebook_folder = 'Colab Notebooks'
notebook1_name = 'Naive_RAG_Milvus_Step2.ipynb'
notebook2_name = 'Advanced_RAG_Step5.ipynb'

# Construct the absolute path
base_drive_path = '/content/drive/My Drive'
absolute_notebook1_path = os.path.join(base_drive_path, notebook_folder, notebook1_name)
absolute_notebook2_path = os.path.join(base_drive_path, notebook_folder, notebook2_name)

print(f"Absolute path for '{notebook1_name}': {absolute_notebook1_path}")
print(f"Absolute path for '{notebook2_name}': {absolute_notebook2_path}")

# You can verify if the files exist at these paths
print(f"Does '{notebook1_name}' exist? {os.path.exists(absolute_notebook1_path)}")
print(f"Does '{notebook2_name}' exist? {os.path.exists(absolute_notebook2_path)}")

Absolute path for 'Naive_RAG_Milvus_Step2.ipynb': /content/drive/My Drive/Colab Notebooks/Naive_RAG_Milvus_Step2.ipynb
Absolute path for 'Advanced_RAG_Step5.ipynb': /content/drive/My Drive/Colab Notebooks/Advanced_RAG_Step5.ipynb
Does 'Naive_RAG_Milvus_Step2.ipynb' exist? True
Does 'Advanced_RAG_Step5.ipynb' exist? True


Then convert notebooks to Python scripts and run.

In [ ]:
# Run notebooks of existing pipeline
# %run "$STEP2_NOTEBOOK" # Removed: Cannot directly run .ipynb files
# %run "$STEP5_NOTEBOOK" # Removed: Cannot directly run .ipynb files
import pymilvus
!pip -q install nbconvert
!jupyter nbconvert --to=python "$STEP2_NOTEBOOK" --output="step2_tmp" --output-dir="/NLX_LLM_Project_2"
!jupyter nbconvert --to=python "$STEP5_NOTEBOOK" --output="step5_tmp" --output-dir="/NLX_LLM_Project_2"

# Run the converted Python scripts
%run "/NLX_LLM_Project_2/step2_tmp.py"
%run "/NLX_LLM_Project_2/step5_tmp.py"


# 6) Confirm conversion
!ls -l /NLX_LLM_Project_2/step*_tmp.py

[NbConvertApp] Making directory /NLX_LLM_Project_2
[NbConvertApp] Converting notebook /content/drive/My Drive/Colab Notebooks/Naive_RAG_Milvus_Step2.ipynb to python
[NbConvertApp] Writing 9189 bytes to /NLX_LLM_Project_2/step2_tmp.py
[NbConvertApp] Converting notebook /content/drive/My Drive/Colab Notebooks/Advanced_RAG_Step5.ipynb to python
[NbConvertApp] Writing 11501 bytes to /NLX_LLM_Project_2/step5_tmp.py


/NLX_LLM_Project_2/step2_tmp.py:48: DeprecationWarning: 'pkgutil.find_loader' is deprecated and slated for removal in Python 3.14; use importlib.util.find_spec() instead
  if pkgutil.find_loader(mod) is None:


OK: datasets
OK: sentence_transformers
OK: pymilvus
OK: numpy
OK: pandas
OK: openai


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/719 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Question: Was Abraham Lincoln the sixteenth President of the United States?


Token indices sequence length is longer than the specified maximum sequence length for this model (555 > 512). Running this sequence through the model will result in indexing errors



Rewrites:
 Is Abraham Lincoln the sixteenth President of the United States?
Abraham Lincoln was the sixteenth President of the United States.
Was Abraham Lincoln the sixteenth President of the United States?

Answer:
 Abraham Lincoln (February 12, 1809 â April 15, 1865) was the sixteenth President of the United States, serving from March 4, 1861 until his assassination.

Citations: [{'id': np.str_('278-0'), 'score': 10.308937072753906}, {'id': np.str_('319-0'), 'score': 8.12199592590332}, {'id': np.str_('198-0'), 'score': -0.4494704604148865}, {'id': np.str_('383-0'), 'score': -0.7347864508628845}, {'id': np.str_('281-0'), 'score': -0.9723390936851501}]
-rw-r--r-- 1 root root  9191 Sep 29 12:08 /NLX_LLM_Project_2/step2_tmp.py
-rw-r--r-- 1 root root 11523 Sep 29 12:08 /NLX_LLM_Project_2/step5_tmp.py


## 3) Define two sets of "answer-context" interface
- Use RAGAS to evaluate answers from naive and advanced pipelines
- For naive pipeline, fetch using top-1 as did in Step 3
- For advanced pipeline, call the answer_advanced function to get a list of answers and context

In [ ]:
# Naive
def naive_answer_and_context(question: str):
    # Use the fallback FAISS retrieval instead of the potentially failing Milvus retrieve function
    hits = retrieve_topk_faiss(question, k=1)   # [(id, text, score)]
    contexts = [hits[0][1]] if hits else []
    # Use the answer_with_context function in Step 2
    try:
        # Note: The original answer_with_context from step2_tmp.py also relies on 'col',
        # so we will use the Flan-T5 pipeline directly for answer generation here
        # as a robust fallback that doesn't depend on Milvus.
        from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
        tok = AutoTokenizer.from_pretrained("google/flan-t5-base")
        mdl = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
        pipe = pipeline("text2text-generation", model=mdl, tokenizer=tok)
        prompt = (
            "Answer strictly using the context. If insufficient, say 'I don't know.'\n\n"
            f"Context:\n{contexts[0] if contexts else ''}\n\nQuestion: {question}\nAnswer:"
        )
        answer = pipe(prompt, max_new_tokens=128)[0]["generated_text"].strip()
    except Exception:
         # Fallback if even the direct pipeline fails (shouldn't happen but good practice)
        answer = "I don't know."
    return answer, contexts

# Advanced
def advanced_answer_and_context(question: str, final_top_k: int = 5, max_ctx_chars: int = 2000):
    # 1) MQE + recall
    candidates, rewrites = gather_candidates(question)
    # 2) Cross-Encoder rerank
    chosen = rerank(question, candidates, top_k=final_top_k)
    # 3) Grounded context
    ctx, citations = build_context_with_citations(chosen, budget=max_ctx_chars)
    # Answer generation
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
    gen_tok = AutoTokenizer.from_pretrained("google/flan-t5-base")
    gen_mdl = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
    gen_pipe = pipeline("text2text-generation", model=gen_mdl, tokenizer=gen_tok)
    prompt = (
        "You are a concise encyclopedia editor. Answer USING ONLY the context; "
        "if insufficient, reply 'I don't know.' Include short inline source tags if helpful.\n\n"
        f"Context:\n{ctx}\n\nQuestion: {question}\nAnswer:"
    )
    answer = gen_pipe(prompt, max_new_tokens=256)[0]["generated_text"].strip()
    # contexts: RAGAS requires a "candidate paragraph list", we use the top-k paragraphs after rerank (remove the labels and give only the plain text)
    contexts = []
    for idx, _score in chosen:
        contexts.append(docs[idx]["text"])
    return answer, contexts

## 4) Prepare for input data for RAGAS

In [ ]:
# ---- Naive retrieval via FAISS (fallback when Milvus `col` is missing) ----
import numpy as np

def retrieve_topk_faiss(query: str, k: int = 1):
    """
    Use Step5's FAISS `index` and `embed_model` to perform a simple vector search.
    Returns: [(idx, text, score), ...]  length = k
    """
    assert 'embed_model' in globals() and 'index' in globals() and 'docs' in globals(), \
        "Step5 resources not found. Ensure step5_tmp.py ran successfully."
    qv = embed_model.encode([query], normalize_embeddings=True).astype("float32")
    D, I = index.search(qv, k)
    hits = []
    for i in range(k):
        idx = int(I[0][i])
        score = float(D[0][i])
        text = docs[idx]["text"]
        hits.append((idx, text, score))
    return hits

# Modify the generation of naive pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
_na_tok = AutoTokenizer.from_pretrained("google/flan-t5-base")
_na_mdl = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
_na_pipe = pipeline("text2text-generation", model=_na_mdl, tokenizer=_na_tok)

def naive_answer_and_context(question: str):
    hits = retrieve_topk_faiss(question, k=1)   # ← Strict top-1 for naive
    contexts = [hits[0][1]] if hits else []
    prompt = (
        "Answer strictly using the context. If insufficient, say 'I don't know.'\n\n"
        f"Context:\n{contexts[0] if contexts else ''}\n\nQuestion: {question}\nAnswer:"
    )
    answer = _na_pipe(prompt, max_new_tokens=128)[0]["generated_text"].strip()
    return answer, contexts

Device set to use cpu


In [ ]:
# ===== Global generator (instantiate once) =====
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

GEN_TOK = AutoTokenizer.from_pretrained("google/flan-t5-base")
GEN_MDL = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
# Auto select device（GPU/CPU）
GEN_PIPE = pipeline(
    task="text2text-generation",
    model=GEN_MDL,
    tokenizer=GEN_TOK,
    device_map="auto"  # or device=0 on GPU / device=-1 on CPU
)

# ===== Truncation helpers  =====
from transformers import AutoTokenizer
from sentence_transformers import CrossEncoder
import numpy as np

# Cross-encoder truncation
_ce_model_id = "cross-encoder/ms-marco-MiniLM-L-6-v2"
_ce_tok = AutoTokenizer.from_pretrained(_ce_model_id)
_RERANKER = CrossEncoder(_ce_model_id, max_length=512)

def _truncate_by_tokens(text: str, tokenizer, max_len: int) -> str:
    ids = tokenizer.encode(text, add_special_tokens=False)
    if len(ids) > max_len:
        ids = ids[:max_len]
    return tokenizer.decode(ids, skip_special_tokens=True)

def rerank(question, candidates, top_k=5, q_budget=32, p_budget=480):
    q_trunc = _truncate_by_tokens(question, _ce_tok, max_len=q_budget)
    pairs, idxs = [], []
    for idx, _, _ in candidates:
        p_text = docs[idx]["text"]
        p_trunc = _truncate_by_tokens(p_text, _ce_tok, max_len=p_budget)
        pairs.append((q_trunc, p_trunc))
        idxs.append(idx)
    if not pairs:
        return []
    scores = _RERANKER.predict(pairs, convert_to_numpy=True, show_progress_bar=False, batch_size=64)
    order = np.argsort(-scores)[:top_k]
    return [(int(idxs[i]), float(scores[i])) for i in order]

# Truncate Flan-T5 output
_FLAN_TOK = GEN_TOK
def _truncate_prompt_for_flan(context: str, question: str, q_budget=48, ctx_budget=460):
    q_trunc  = _truncate_by_tokens(question, _FLAN_TOK, max_len=q_budget)
    ctx_trunc = _truncate_by_tokens(context,  _FLAN_TOK, max_len=ctx_budget)
    return ctx_trunc, q_trunc

def persona_prompt(context, question):
    ctx_trunc, q_trunc = _truncate_prompt_for_flan(context, question)
    return (
        "You are a concise encyclopedia editor. Answer USING ONLY the context; "
        "if insufficient, reply 'I don't know.' Include short inline source tags if helpful.\n\n"
        f"Context:\n{ctx_trunc}\n\nQuestion: {q_trunc}\nAnswer:"
    )

# ===== Corrected advanced pipeline function =====
def advanced_answer_and_context(question: str, final_top_k: int = 5, max_ctx_chars: int = 2000):
    # 1) MQE + recall
    candidates, rewrites = gather_candidates(question)
    # 2) Cross-encoder rerank
    chosen = rerank(question, candidates, top_k=final_top_k)
    # 3) build context with citations
    ctx, citations = build_context_with_citations(chosen, budget=max_ctx_chars)
    # 4) chunk prompts
    prompt = persona_prompt(ctx, question)
    ans = GEN_PIPE(prompt, max_new_tokens=256)[0]["generated_text"].strip()
    # return text for RAGAS input
    contexts = [docs[idx]["text"] for idx, _ in chosen]
    return ans, contexts

Device set to use cpu


In [ ]:
# --- 1) Generate evidence for reference in docs ---
import re
import numpy as np

def _find_reference_passages(question: str, gold_answer: str, k: int = 1):
    """
    Return reference=[list[str]] for RAGAS input
    Search for chunk of ground truth
    """
    refs = []
    ga = (gold_answer or "").strip()
    if ga:
        pat = re.escape(ga)
        for d in docs:
            txt = d["text"]
            if re.search(pat, txt, flags=re.IGNORECASE):
                refs.append(txt)
                if len(refs) >= k:
                    break

    # If no reference found: use embedding vector to search
    if not refs:
        qv = embed_model.encode([question], normalize_embeddings=True).astype("float32")
        D, I = index.search(qv, max(k, 5))
        # choose top k for evidence
        for i in range(min(k, I.shape[1])):
            idx = int(I[0][i])
            refs.append(docs[idx]["text"])


    return [refs]  # [[ref1, ref2, ...]]




## 5) Run RAGAS evaluation
- Load `question-answer` subset first
- Check presence of Naive/Advanced pipeline
- Use a local embedding adaptor to calculate same metrics if RAGAS fails
- Construct input for RAGAS evaluation

In [15]:
# ====================================================================================
# Step 6 — RAGAS Evaluation (Offline, No OpenAI) — modified for cases without LLM APIs
# ====================================================================================
!pip -q install "ragas>=0.1.9" datasets evaluate pandas transformers sentence-transformers

import os, re, importlib, numpy as np, pandas as pd
os.environ["RAGAS_DISABLE_TELEMETRY"] = "1"

# ---------------------------
# 1) Load Q-A
# ---------------------------
from datasets import load_dataset, Dataset
if "qa" not in globals():
    ds = load_dataset("rag-datasets/rag-mini-wikipedia", "question-answer")
    qa = ds["test"]
if "M" not in globals():
    M = min(50, len(qa))
print(f"[Info] QA size={len(qa)}, M={M}")

# -----------------------------------------
# 2) Check for Step2/5
# -----------------------------------------
need = [("embed_model","SentenceTransformer"), ("index","FAISS index"), ("docs","list of chunks")]
missing = [n for n,_ in need if n not in globals()]
assert not missing, f"Missing upstream resources: {missing}. Make sure Step2/Step5 ran."

# --------------------------------------------------
# 3) Recreate naive pipeline if missing
# --------------------------------------------------
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

if "naive_answer_and_context" not in globals():
    _na_tok = AutoTokenizer.from_pretrained("google/flan-t5-base")
    _na_mdl = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
    _na_pipe = pipeline("text2text-generation", model=_na_mdl, tokenizer=_na_tok, device_map="auto")

    def _retrieve_topk_faiss(query: str, k: int = 1):
        qv = embed_model.encode([query], normalize_embeddings=True).astype("float32")
        D, I = index.search(qv, k)
        hits = []
        for i in range(k):
            idx = int(I[0][i])
            score = float(D[0][i])
            hits.append((idx, docs[idx]["text"], score))
        return hits

    def naive_answer_and_context(question: str):
        hits = _retrieve_topk_faiss(question, k=1)
        contexts = [hits[0][1]] if hits else [""]
        prompt = (
            "Answer strictly using the context. If insufficient, say 'I don't know.'\n\n"
            f"Context:\n{contexts[0]}\n\nQuestion: {question}\nAnswer:"
        )
        ans = _na_pipe(prompt, max_new_tokens=128)[0]["generated_text"].strip()
        return ans, contexts

if "advanced_answer_and_context" not in globals():
    raise RuntimeError("Step5 (Advanced RAG) must be executed first to define advanced_answer_and_context.")

# ------------------------------------------------------
# 4) Local embeddings adaptor
# ------------------------------------------------------
from sentence_transformers import SentenceTransformer

class LocalSTEmbeddings:
    def __init__(self, model_id="sentence-transformers/all-MiniLM-L6-v2"):
        self.model = SentenceTransformer(model_id)
    def embed_documents(self, texts):
        vecs = self.model.encode(list(texts), normalize_embeddings=True)
        return [v.tolist() for v in np.array(vecs, dtype="float32")]
    def embed_query(self, text):
        vec = self.model.encode([text], normalize_embeddings=True)[0]
        return np.array(vec, dtype="float32").tolist()
    async def aembed_documents(self, texts):
        return self.embed_documents(texts)
    async def aembed_query(self, text):
        return self.embed_query(text)

embeddings = LocalSTEmbeddings("sentence-transformers/all-MiniLM-L6-v2")

# -------------------------------------------------------------------------------------
# 5) Inhibit use of OpenAI functionalities in RAGAS since we do not have valid API key
# -------------------------------------------------------------------------------------
def _disabled_llm_factory(*args, **kwargs):
    return None
for mod in ["ragas.llms.base", "ragas.evaluation", "ragas.metrics.base", "ragas.metrics"]:
    try:
        m = importlib.import_module(mod)
        if hasattr(m, "llm_factory"):
            setattr(m, "llm_factory", _disabled_llm_factory)
    except Exception:
        pass

# -------------------------------------------------
# 6) Minimum local LLM adaptor
# -------------------------------------------------


from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# Initialize Flan-T5 locally
_llm_tok = AutoTokenizer.from_pretrained("google/flan-t5-base")
_llm_mdl = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
_llm_pipe = pipeline("text2text-generation", model=_llm_mdl, tokenizer=_llm_tok, device_map="auto")

# ——— drop-in replacement ———
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

_llm_tok = AutoTokenizer.from_pretrained("google/flan-t5-base")
_llm_mdl = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
_llm_pipe = pipeline("text2text-generation", model=_llm_mdl, tokenizer=_llm_tok, device_map="auto")

class SimpleHFLLM:
    """
    Minimal RAGAS-compatible local LLM adapter.
    Implements: name, set_run_config, generate/agenerate, generate_prompt/agenerate_prompt.
    """
    name = "local-hf-flan-t5"

    def __init__(self, gen_pipe, default_max_new_tokens: int = 64):
        self.pipe = gen_pipe
        self._run_config = None
        self._default_max_new_tokens = default_max_new_tokens

    def set_run_config(self, run_config):
        self._run_config = run_config

    # ---------- helpers ----------
    def _pick_max_new_tokens(self, kwargs):
        if "max_new_tokens" in kwargs and kwargs["max_new_tokens"] is not None:
            return int(kwargs["max_new_tokens"])
        if self._run_config is not None and hasattr(self._run_config, "max_new_tokens"):
            try:
                return int(self._run_config.max_new_tokens)
            except Exception:
                pass
        return self._default_max_new_tokens

    def _to_text(self, prompt_obj):

        try:
            # langchain / ragas prompt objects often support format / to_string
            if hasattr(prompt_obj, "to_string"):
                return prompt_obj.to_string()
            if hasattr(prompt_obj, "text"):
                return str(prompt_obj.text)
        except Exception:
            pass
        return str(prompt_obj)

    # ---------- APIs expected by RAGAS ----------
    def generate(self, prompts, **kwargs):
        max_new_tokens = self._pick_max_new_tokens(kwargs)
        outs = []
        for p in prompts:
            text = self._to_text(p)
            o = self.pipe(text, max_new_tokens=max_new_tokens)[0]["generated_text"]
            outs.append(o)
        return outs

    async def agenerate(self, prompts, **kwargs):
        return self.generate(prompts, **kwargs)

    # Some ragas versions call these prompt-specific methods
    def generate_prompt(self, prompts, **kwargs):
        return self.generate(prompts, **kwargs)

    async def agenerate_prompt(self, prompts, **kwargs):
        return self.agenerate(prompts, **kwargs)


local_llm = SimpleHFLLM(_llm_pipe)




# -------------------------------------------------
# 7) Construct input for RAGAS
# -------------------------------------------------
def _best_reference_passage(question: str, gold_answer: str) -> str:
    ga = (gold_answer or "").strip()
    if ga:
        pat = re.escape(ga)
        for d in docs:
            t = d["text"]
            if re.search(pat, t, flags=re.IGNORECASE):
                return t
    qv = embed_model.encode([question], normalize_embeddings=True).astype("float32")
    D, I = index.search(qv, 1)
    idx = int(I[0][0])
    return docs[idx]["text"]

def _safe_append(sample_list, item):
    required = ("question","answer","contexts","ground_truths","reference")
    ok = all(k in item for k in required)
    ok = ok and isinstance(item["question"], str) \
             and isinstance(item["answer"], str) \
             and isinstance(item["contexts"], list) \
             and isinstance(item["ground_truths"], list) \
             and isinstance(item["reference"], str)
    if ok:
        sample_list.append(item)
    return ok

def build_ragas_inputs(pipeline="naive") -> Dataset:
    samples, skipped = [], 0
    for i in range(M):
        q = qa[i]["question"]
        gt = qa[i]["answer"] if "answer" in qa[i] else qa[i]["answers"]
        gt_text = gt if isinstance(gt, str) else gt[0]
        try:
            if pipeline == "naive":
                a, ctxs = naive_answer_and_context(q)
            else:
                a, ctxs = advanced_answer_and_context(q, final_top_k=5, max_ctx_chars=2000)

            if not isinstance(ctxs, list):
                ctxs = [str(ctxs)] if ctxs else [""]

            ref = _best_reference_passage(q, gt_text)
            item = {
                "question": q,
                "answer": a if isinstance(a, str) else str(a),
                "contexts": [str(c) for c in ctxs] if ctxs else [""],
                "ground_truths": [gt_text],
                "reference": ref if isinstance(ref, str) else str(ref),
            }
            if not _safe_append(samples, item):
                skipped += 1
        except Exception:
            skipped += 1
            continue

    if not samples:
        raise RuntimeError("No valid samples built for RAGAS.")
    print(f"Built {len(samples)} samples; skipped {skipped}")
    return Dataset.from_list(samples)

ds_naive = build_ragas_inputs("naive")
ds_adv   = build_ragas_inputs("advanced")
print("Sample (naive):", ds_naive[0])
print("Sample (advanced):", ds_adv[0])






[Info] QA size=918, M=50


Device set to use cpu
Device set to use cpu


Built 50 samples; skipped 0
Built 50 samples; skipped 0
Sample (naive): {'question': 'Was Abraham Lincoln the sixteenth President of the United States?', 'answer': "I don't know", 'contexts': ['Young Abraham Lincoln'], 'ground_truths': ['yes'], 'reference': "At the age of twenty, in 1812, at the end of his apprenticeship, Faraday attended lectures by the eminent English chemist and physicist Humphry Davy of the Royal Institution and Royal Society, and John Tatum, founder of the City Philosophical Society. Many tickets for these lectures were given to Faraday by William Dance (one of the founders of the Royal Philharmonic Society). Afterwards, Faraday sent Davy a three hundred page book based on notes taken during the lectures. Davy's reply was immediate, kind, and favorable. When Davy damaged his eyesight in an accident with nitrogen trichloride, "}
Sample (advanced): {'question': 'Was Abraham Lincoln the sixteenth President of the United States?', 'answer': 'Abraham Lincoln (February 

In [20]:
# =========================
# Cell 16 — Local ID-matching Evaluation (no RAGAS metrics, no LLM)
# =========================
import numpy as np
import pandas as pd
from datasets import Dataset

# 0) Assert context
assert "ds_naive" in globals() and "ds_adv" in globals(), "Needs ds_naive / ds_adv first"
assert "docs" in globals(), "Needs docs (chunk list with text and/or id)"
assert "embed_model" in globals() and "index" in globals(), "Needs embed_model + FAISS index"

# 1) Map text->id
_text2id = {}
for i, d in enumerate(docs):
    tid = str(d.get("id", i))
    _text2id[d["text"]] = tid

def _map_text_to_id(txt: str) -> str:
    if not txt:
        return None
    if txt in _text2id:
        return _text2id[txt]
    # In case length exceeds model maximum: vector search
    snippet = txt[:1200]
    vec = embed_model.encode([snippet], normalize_embeddings=True).astype("float32")
    D, I = index.search(vec, 1)
    return str(I[0][0])

def _build_id_rows(ds: Dataset):
    rows = []
    for r in ds:
        ctx_texts = r.get("contexts") or []
        ref_text  = r.get("reference", "")
        ctx_ids = [ _map_text_to_id(t) for t in ctx_texts if t ]
        ref_id  = _map_text_to_id(ref_text)
        # Avoid outliers
        if not ctx_ids or ref_id is None:
            continue
        rows.append({
            "retrieved_ids": ctx_ids,
            "reference_ids": [ref_id],
        })
    if not rows:
        raise RuntimeError("No valid samples created, please check compatibility of contexts/reference with docs")
    return rows

rows_naive = _build_id_rows(ds_naive)
rows_adv   = _build_id_rows(ds_adv)

# 2) ID-based Precision / Recall（Macro & Micro）
def _score(rows):
    # for each sample:
    #  precision_i = |retrieved ∩ reference| / |retrieved|
    #  recall_i    = |retrieved ∩ reference| / |reference|
    # Macro: average; Micro: global
    precisions, recalls = [], []
    inter_sum, ret_sum, ref_sum = 0, 0, 0
    for r in rows:
        ret = set(r["retrieved_ids"])
        ref = set(r["reference_ids"])
        inter = len(ret & ref)
        p_i = inter / len(ret) if len(ret) > 0 else 0.0
        r_i = inter / len(ref) if len(ref) > 0 else 0.0
        precisions.append(p_i)
        recalls.append(r_i)
        inter_sum += inter
        ret_sum   += len(ret)
        ref_sum   += len(ref)
    macro_p = float(np.mean(precisions)) if precisions else 0.0
    macro_r = float(np.mean(recalls))    if recalls    else 0.0
    micro_p = float(inter_sum / ret_sum) if ret_sum > 0 else 0.0
    micro_r = float(inter_sum / ref_sum) if ref_sum > 0 else 0.0
    return {
        "context_precision_macro": macro_p,
        "context_recall_macro": macro_r,
        "context_precision_micro": micro_p,
        "context_recall_micro": micro_r,
        "n_samples_scored": len(rows),
    }

scores_naive = _score(rows_naive)
scores_adv   = _score(rows_adv)

# 3) Final results
df_cmp = pd.DataFrame([{"system":"naive", **scores_naive},
                       {"system":"advanced", **scores_adv}]).set_index("system")
display(df_cmp)




,context_precision_macro,context_recall_macro,context_precision_micro,context_recall_micro,n_samples_scored
system,,,,,
naive,0.160,0.16,0.160,0.16,50
advanced,0.048,0.24,0.048,0.24,50


## 6) Save results as .csv file

In [25]:

SAVE_CSV = "/content/drive/MyDrive/NLX_LLM_Project_2/step6_ragas_results.csv"
df_cmp.to_csv(SAVE_CSV, index=False)
print("Saved to:", SAVE_CSV)


Saved to: /content/drive/MyDrive/NLX_LLM_Project_2/step6_ragas_results.csv
